In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# load data 
from src.data_loader import load_data

# load the data 
df = load_data("/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv")

# basic info 
print("dataset shape:", df.shape)
print("\ndata types:")
print(df.dtypes)
print("\nmissing values:")
print(df.isnull().sum())

# address the target var

In [ ]:
# Check autism diagnosis columns (should have values 1, 2, or 3)
print("Autism diagnosis columns:")
for col in [col for col in df.columns if 'autism_diagnosis' in col]:
    print(f"{col}: {df[col].value_counts().to_dict()}")
    print(f"Missing: {df[col].isnull().sum()}")
    print("---")

# Check regular diagnosis columns (should have value 2 for autism)
print("\nRegular diagnosis columns:")
for col in [col for col in df.columns if col.startswith('diagnosis_') and not 'autism' in col]:
    print(f"{col}: {df[col].value_counts().to_dict()}")
    print(f"Missing: {df[col].isnull().sum()}")
    print("---")




In [ ]:
# check what diagnosis columns actually exist 
print("all columns containing 'diagnosis':")
diagnosis_related = [col for col in df.columns if 'diagnosis' in col]
print(diagnosis_related)

print("\nall autism diagnosis columns:")
autism_diag = [col for col in df.columns if 'autism_diagnosis' in col]
print(autism_diag)


In [ ]:
# get the correct columns names 
autism_cols = [col for col in df.columns if 'autism_diagnosis' in col]
diagnosis_cols = [col for col in df.columns if col.startswith('diagnosis_') and not 'autism' in col]

print(f"autism columns found: {autism_cols}")
print(f"diagnosis columns found: {diagnosis_cols}")

# method 1 autism diagnosis columns 
if autism_cols:
    autism_from_specific = df[autism_cols].fillna(0).ge(1).any(axis=1)
else:
    autism_from_specific = pd.Series([False] * len(df))

# method 2 regular diagnosis columns 
if diagnosis_cols:
    autism_from_general = df[diagnosis_cols].fillna(0).eq(2).any(axis=1)
else: 
    autism_from_general = pd.Series([False] * len(df))

df['autism_target'] = (autism_from_specific | autism_from_general).astype(int)

# creating target var 

In [ ]:
# get columns dynamically from your datafram
autism_cols = [col for col in df.columns if 'autism_diagnosis' in col]
diagnosis_cols = [col for col in df.columns if col.startswith('diagnosis_') and not 'autism' in col]

# method 1 autism fiagnosis columns
autism_from_specific = df[autism_cols].fillna(0).ge(1).any(axis=1)

# method 2 regular diagnosis columns 
autism_from_general = df[diagnosis_cols].fillna(0).eq(2).any(axis=1)

# combined target 
df['autism_target'] = (autism_from_specific | autism_from_general).astype(int)

# analysis of target var 
print("combined target variable analysis:")
print("total cases: {len(df)}")
print(f"autism cases: {df['autism_target'].sum()}")
print(f"non-autism cases: {len(df) - df['autism_target'].sum()}")
print(f"autism percentage: {(df['autism_target'].sum() / len(df)) * 100:.2f}%")

# check for class imbalance 
print(f"\nclass imbalance:")
print(f"autism: {df['autism_target'].sum()} ({df['autism_target'].mean()*100:.1f}%)")
print(f"non-autism: {(df['autism_target'] == 0).sum()} ({(1-df['autism_target'].mean())*100:.1f}%)")

# EDA


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# visualise class imbalance
sns.countplot(x='autism_target', data=df)
plt.title('class balance: autism vs non-autism')
plt.xticks([0, 1], ['non-autism', 'autism'])
plt.show()

# age distribution 
sns.histplot(data=df, x='age', hue='autism_target', bins=30, kde=True, stat='density')
plt.title('age distribution by autism status')
plt.show()

# sex distribution 
sex_map = {0: 'male', 1: 'female'}
df['sex_label'] = df['sex'].map(sex_map)  # create a new column

sns.countplot(x='sex', hue='autism_target', data=df)
plt.title('Sex by Autism Diagnosis')
plt.show()

In [ ]:
# questionnaire scores
import matplotlib.pyplot as plt
import seaborn as sns

#list all questionnaire columns
questionnaire_cols = [col for col in df.columns if any(q in col for q in ['spq_', 'eq_', 'spqr_', 'aq_'])]

df_melted = df.melt(id_vars='autism_target', value_vars=questionnaire_cols,
                             var_name='questionnaire', value_name='score')

plt.figure(figsize=(16, 6))
sns.boxplot(x='questionnaire', y='score', hue='autism_target', data=df_melted)
plt.title('distribution of questionnaire scores by autism diagnosis')
plt.xticks(rotation=90)
plt.legend(title='autism')
plt.show()

#plot violins
plt.figure(figsize=(16, 6))
sns.violinplot(x='questionnaire', y='score', hue='autism_target', data=df_melted, split=True)
plt.title('violin plot of questionnaire scores by autism diagnosis')
plt.xticks(rotation=90)
plt.legend(title='autism')
plt.show()

# facegrid 
g = sns.FacetGrid(df_melted, col='questionnaire', col_wrap=5, height=3, sharey=False)
g.map(sns.boxplot, "autism_target", "score", order=[0, 1])
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('boxplot of questionnaire scores by autism diagnosis')
plt.show()

# heatmap of mean differences 
means = df.groupby('autism_target')[questionnaire_cols].mean().T
means['diff'] = means[1] - means[0]
plt.figure(figsize=(10, 1))
sns.heatmap(means[['diff']].T, annot=True, cmap='coolwarm', center=0)
plt.title('mean score difference (autism - non-autism)')
plt.yticks(rotation=0)
plt.show()
